# Master DSLS / Programming 3 / Assignment 6
# Final Assignment

## Introduction
This is the final for programming 3. In this assignment, I will develop scikit-learn machine learning models to predict the function of the proteins in the specific dataset. This model will use small InterPro_annotations_accession to predict large InterPro_annotations_accession.
The definition of small InterPro_annotations_accession and large InterPro_annotations_accession is defined as below:

If InterPro_annotations_accession's feature length(Stop_location-Start_location) / Sequence_length > 0.9, it is large InterPro_annotations_accession.

Otherwise, it is a small InterPro_annotations_accession.

We can briefly rewrite as:

            |(Stop - Start)|/Sequence >  0.9 --> Large

            |(Stop - Start)|/Sequence <= 0.9 --> small

I will also check the "bias" and "noise" that does not make sense from the dataset.

ie. lines(-) from the TSV file which don't contain InterPRO numbers

ie. proteins which don't have a large feature (according to the criteria above)

## 1. Goal

The goal of this assignment is to predict large InterPro_annotations_accession by small InterPro_annotations_accession.

I will use the dataset from /data/dataprocessing/interproscan/all_bacilli.tsv file on assemblix2012 and assemblix2019. However, this file contains ~4,200,000 protein annotations, so I will put a subset of all_bacilli.tsv on GitHub and on local for code testing.

In [1]:
# Output format : https://interproscan-docs.readthedocs.io/en/latest/OutputFormats.html
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,FloatType
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
import numpy as np
import pandas as pd
import warnings
import time
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 

In [2]:
# Create a df by PySpark
schema = StructType([
    StructField("Protein_accession", StringType(), True),
    StructField("Sequence_MD5_digest", StringType(), True),
    StructField("Sequence_length", IntegerType(), True),
    StructField("Analysis", StringType(), True),
    StructField("Signature_accession", StringType(), True),
    StructField("Signature_description", StringType(), True),
    StructField("Start_location", IntegerType(), True),
    StructField("Stop_location", IntegerType(), True),
    StructField("Score", FloatType(), True),
    StructField("Status", StringType(), True),
    StructField("Date", StringType(), True),
    StructField("InterPro_annotations_accession", StringType(), True),
    StructField("InterPro_annotations_description", StringType(), True),
    StructField("GO_annotations", StringType(), True),
    StructField("Pathways_annotations", StringType(), True)])
# path = "/data/dataprocessing/interproscan/all_bacilli.tsv"
path = "all_bacilli.tsv"
spark = SparkSession.builder.master("local[16]")\
                            .config('spark.driver.memory', '128g')\
                            .config('spark.executor.memory', '128g')\
                            .config("spark.sql.debug.maxToStringFields","100")\
                            .appName("InterPro").getOrCreate()
df = spark.read.option("sep","\t").option("header","False").csv(path,schema=schema)

22/09/25 21:13:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df.printSchema()

root
 |-- Protein_accession: string (nullable = true)
 |-- Sequence_MD5_digest: string (nullable = true)
 |-- Sequence_length: integer (nullable = true)
 |-- Analysis: string (nullable = true)
 |-- Signature_accession: string (nullable = true)
 |-- Signature_description: string (nullable = true)
 |-- Start_location: integer (nullable = true)
 |-- Stop_location: integer (nullable = true)
 |-- Score: float (nullable = true)
 |-- Status: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- InterPro_annotations_accession: string (nullable = true)
 |-- InterPro_annotations_description: string (nullable = true)
 |-- GO_annotations: string (nullable = true)
 |-- Pathways_annotations: string (nullable = true)



In [4]:
# One line code
# remove InterPro_annotations_accession == "-"
# get the length of protein
# get the ratio to distinguish them to large and small InterPro_annotations_accession
# 1 for large, 0 for small InterPro_annotations_accession
start = time.time()
df = df.filter(df.InterPro_annotations_accession != "-")\
        .withColumn("Length", abs(df["Stop_location"] - df["Start_location"]))\
        .withColumn("Ratio", (abs(df["Stop_location"] - df["Start_location"])/df["Sequence_length"]))\
        .withColumn("Size", when((abs(df["Stop_location"] - df["Start_location"])/df["Sequence_length"])>0.9,1).otherwise(0))
# get the intersection to make sure there is a match of large and small InterPro_annotations_accession(at least one large and one small InterPro_annotations_accession)
intersection = df.filter(df.Size == 0).select("Protein_accession").intersect(df.filter(df.Size == 1).select("Protein_accession"))
intersection_df = intersection.join(df,["Protein_accession"])
# get the number of small InterPro_annotations_accession in each Protein_accession
small_df = intersection_df.filter(df.Size == 0).groupBy(["Protein_accession"]).pivot("InterPro_annotations_accession").count()
# There are several InterPro_annotations_accession with the same Protein_accession. I only choose the largest one.
large_df = intersection_df.filter(df.Size == 1).groupby(["Protein_accession"]).agg(max("Ratio").alias("Ratio"))
large_df = large_df.join(intersection_df,["Protein_accession","Ratio"],"inner").dropDuplicates(["Protein_accession"])
# Drop the useless columns
columns = ("Sequence_MD5_digest","Analysis","Signature_accession","Signature_description",
        "Score","Status","Date","InterPro_annotations_description","GO_annotations",
        "Pathways_annotations","Ratio","Size","Stop_location","Start_location","Sequence_length","Length")
large_df = large_df.drop(*columns)


In [5]:
# Create the df for ML
ML_df = large_df.join(small_df,["Protein_accession"]).fillna(0)

In [6]:
# Setup X,y and split it
y = ML_df.select("InterPro_annotations_accession")
X = ML_df.select(ML_df.columns[2:])
y = np.array(y.collect())
X = np.array(X.collect())
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size)

22/09/25 21:14:11 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
# Modeling
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)
# performing predictions on the test dataset
y_pred = rfc.predict(X_test)
# using metrics module for accuracy calculation
accuracy=metrics.accuracy_score(y_test, y_pred)
print(accuracy)
end = time.time()
print(end-start)

0.0
61.408486127853394


In [8]:
import pickle
# file = '/students/2021-2022/master/Kai_DSLS/randforest.pkl'
file = "randforest.pkl"
pickle.dump(rfc, open(file, 'wb'))
# file_Xtrain = '/students/2021-2022/master/Kai_DSLS/X_train.pkl'
file_Xtrain = "./X_train.pkl"
ML_df.toPandas().to_pickle(file_Xtrain)